### __Group 9: Mazarine Tulp, Floor Halkes, and Kyra Jongman__

07-11-2024

### __Import Packages__

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [3]:
from nlp.preprocessing_functions import *

[nltk_data] Downloading package punkt to /home/floor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/floor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/floor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### __Load Data__

In [4]:
# OPEN CSV FILES

# This dataset includes news and opinion articles about various companies
df = pd.read_csv('data/raw/us_equities_news_dataset.csv')

# DROP DUPLICATES

df_filtered = df.drop_duplicates(subset=['content'])

# DROP ROWS WHERE 'CONTENT' columns has NANS

df_filtered = df_filtered.dropna(subset=['content'])

# FILTER DATAFRAME ON CONTENT OF EITHER 'NVDA' OR 'NVIDIA'

# Check if each row in 'content' column contains the words "nvda" or "nvidia
df_filtered_on_content = df_filtered[df_filtered['content'].fillna('').str.contains(
    r'\bnvda\b|\bnvidia\b', case=False, regex=True)]
print(f"The amount of articles is (both news and opinions): {len(df_filtered_on_content)}")

The amount of articles is (both news and opinions): 3436


In [33]:
df_filtered_on_content.head(5)

,id,ticker,title,category,content,release_date,provider,url,article_id
24,221539,NIO,A Central Bank War Just Started And Its Good F...,opinion,ECB Effects\nThe move in the euro was huge fa...,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687
32,221547,NIO,6 Stocks To Watch Nivida Could Be Falling,opinion,6 Stocks To Watch March 6 Trading Session\nSt...,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931
57,221572,NIO,Stocks Dow Drops Nearly 400 Points as Apple ...,news,Investing com A rout in Apple and Facebook ...,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042
78,221593,UBER,The Zacks Analyst Blog Highlights Advanced Mi...,opinion,For Immediate ReleaseChicago IL January 13 ...,2020-01-12,Zacks Investment Research,https://www.investing.com/analysis/the-zacks-a...,200498277
82,221597,UBER,The Best Of CES 2020 Revised,opinion,With 4 500 companies bringing their innovation...,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164


### __Preprocessing Steps:__

In [5]:
# PREPROCESSING CONTENT COLUMN

# Convert to lowercase
df_filtered_on_content.loc[:,'preprocessed_content'] = convert_to_lowercase(df_filtered_on_content['content'])

/tmp/ipykernel_31641/3072511301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_on_content.loc[:,'preprocessed_content'] = convert_to_lowercase(df_filtered_on_content['content'])


In [6]:
# Remove URLs
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_all_urls(df_filtered_on_content['preprocessed_content'])

In [7]:
# Remove all words that are note characters
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_non_words_characters(df_filtered_on_content['preprocessed_content'])

In [8]:
# Remove digits
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_digits(df_filtered_on_content['preprocessed_content'])

In [9]:
# Tokenize words
df_filtered_on_content.loc[:, 'preprocessed_content'] = tokenize_words(df_filtered_on_content['preprocessed_content'])

In [10]:
# Remove stopwords
df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_stopwords(df_filtered_on_content['preprocessed_content'])

In [74]:
# Lemmatize words
df_filtered_on_content.loc[:, 'preprocessed_content'] = lemmatize_words(df_filtered_on_content['preprocessed_content'])

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>

In [12]:
# ik had het even in losse cellen gezet om te testen maar kan ook weer zo in deze cel bij elkaar

# # PREPROCESSING CONTENT COLUMN

# # Convert to lowercase
# df_filtered_on_content.loc[:, 'preprocessed_content'] = convert_to_lowercase(df_filtered_on_content['content'])

# # Remove URLs
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_all_urls(df_filtered_on_content['preprocessed_content'])

# # Remove all words that are note characters
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_non_words_characters(df_filtered_on_content['preprocessed_content'])

# # Remove digits
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_digits(df_filtered_on_content['preprocessed_content'])

# # Tokenize words
# df_filtered_on_content.loc[:, 'preprocessed_content'] = tokenize_words(df_filtered_on_content['preprocessed_content'])

# # Remove stopwords
# df_filtered_on_content.loc[:, 'preprocessed_content'] = remove_stopwords(df_filtered_on_content['preprocessed_content'])

# # Lemmatize words
# df_filtered_on_content.loc[:, 'preprocessed_content'] = lemmatize_words(df_filtered_on_content['preprocessed_content'])

# df_filtered_on_content.head(5)

[nltk_data] Downloading package punkt to /home/floor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/floor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/floor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### __Remove least used and most used words:__

In [47]:
def get_word_frequency(series:pd.Series):
    all_words_after_preprocessing = [l for ls in series for l in ls]
    counter = Counter(all_words_after_preprocessing)
    word_count = counter.most_common()
    return word_count

In [48]:
total_number_of_words = len(get_word_frequency(df_filtered_on_content['preprocessed_content']))
print(f"Total number of words in corpus is {total_number_of_words}")

Total number of words in corpus is 32200


In [49]:
word_frequency = get_word_frequency(df_filtered_on_content['preprocessed_content'])
word_frequency

[('nasdaq', 14843),
 ('year', 13835),
 ('company', 11343),
 ('zacks', 11194),
 ('earnings', 9880),
 ('nvidia', 8932),
 ('stocks', 8369),
 ('market', 8238),
 ('quarter', 7770),
 ('growth', 7333),
 ('stock', 7058),
 ('also', 6241),
 ('nyse', 6181),
 ('billion', 6146),
 ('new', 5067),
 ('last', 4737),
 ('rank', 4715),
 ('million', 4714),
 ('inc', 4600),
 ('technology', 4583),
 ('shares', 4471),
 ('expected', 4268),
 ('u', 4168),
 ('companies', 4085),
 ('investors', 4060),
 ('p', 4041),
 ('nvda', 3958),
 ('revenues', 3900),
 ('week', 3829),
 ('data', 3785),
 ('share', 3728),
 ('one', 3678),
 ('per', 3637),
 ('strong', 3628),
 ('buy', 3627),
 ('top', 3316),
 ('time', 3315),
 ('revenue', 3285),
 ('trade', 3272),
 ('consensus', 3235),
 ('intel', 3222),
 ('like', 3214),
 ('estimate', 3169),
 ('see', 3086),
 ('china', 3081),
 ('said', 3043),
 ('industry', 2992),
 ('investment', 2986),
 ('index', 2958),
 ('high', 2934),
 ('could', 2920),
 ('higher', 2918),
 ('apple', 2839),
 ('report', 2839),
 (

In [53]:
most_common_words = word_frequency[:20]
print(f"20 most common words are \n {most_common_words}")

20 most common words are 
 [('nasdaq', 14843), ('year', 13835), ('company', 11343), ('zacks', 11194), ('earnings', 9880), ('nvidia', 8932), ('stocks', 8369), ('market', 8238), ('quarter', 7770), ('growth', 7333), ('stock', 7058), ('also', 6241), ('nyse', 6181), ('billion', 6146), ('new', 5067), ('last', 4737), ('rank', 4715), ('million', 4714), ('inc', 4600), ('technology', 4583)]


In [56]:
def get_number_of_words_with_up_to_n_times_present(word_frequency, n):
    least_common = sum(1 for t in word_frequency if 1 <= t[1] <= n)
    return least_common

get_number_of_words_with_up_to_n_times_present(word_frequency, 5)

21079

In [75]:
used_5_times = [word for word in word_frequency if word[1] == 5]
used_5_times

[('bunds', 5),
 ('deflate', 5),
 ('refuge', 5),
 ('folding', 5),
 ('thinkpad', 5),
 ('converts', 5),
 ('brush', 5),
 ('neon', 5),
 ('powervision', 5),
 ('aerial', 5),
 ('avoidance', 5),
 ('debuting', 5),
 ('attaching', 5),
 ('achievable', 5),
 ('sdks', 5),
 ('localization', 5),
 ('impractical', 5),
 ('awful', 5),
 ('weathered', 5),
 ('tired', 5),
 ('accompanying', 5),
 ('meli', 5),
 ('tourists', 5),
 ('venues', 5),
 ('emission', 5),
 ('humanoid', 5),
 ('juggernaut', 5),
 ('ridesharing', 5),
 ('notifications', 5),
 ('cybertruck', 5),
 ('ingest', 5),
 ('formats', 5),
 ('pocketed', 5),
 ('grants', 5),
 ('neumann', 5),
 ('plowing', 5),
 ('fisv', 5),
 ('claw', 5),
 ('upping', 5),
 ('kilometers', 5),
 ('paced', 5),
 ('ebbed', 5),
 ('ebbs', 5),
 ('noisy', 5),
 ('mojo', 5),
 ('mightily', 5),
 ('reserved', 5),
 ('defective', 5),
 ('woo', 5),
 ('coupon', 5),
 ('academy', 5),
 ('clearance', 5),
 ('punitive', 5),
 ('chipped', 5),
 ('outdone', 5),
 ('ix', 5),
 ('nussey', 5),
 ('improbable', 5),
 ('

In [67]:
least_common = get_word_frequency(df_filtered_on_content['preprocessed_content'])[:-21079-1:-1]
least_common

[('mbfi', 1),
 ('csl', 1),
 ('nus', 1),
 ('growththere', 1),
 ('consensusrose', 1),
 ('uwti', 1),
 ('trgp', 1),
 ('targa', 1),
 ('lgcy', 1),
 ('oriental', 1),
 ('derm', 1),
 ('dermira', 1),
 ('thereabouts', 1),
 ('codenames', 1),
 ('sheetadvanced', 1),
 ('agreementduring', 1),
 ('headquartersduring', 1),
 ('revenuesadvanced', 1),
 ('sharesplunged', 1),
 ('fal', 1),
 ('objectively', 1),
 ('jasper', 1),
 ('unelected', 1),
 ('hypothesized', 1),
 ('linelove', 1),
 ('reviewearlier', 1),
 ('derivativeslast', 1),
 ('explainedbitcoin', 1),
 ('snatch', 1),
 ('monumental', 1),
 ('riskiness', 1),
 ('upheavals', 1),
 ('cafe', 1),
 ('mafia', 1),
 ('frontierville', 1),
 ('farmville', 1),
 ('cityville', 1),
 ('etfswhile', 1),
 ('opportunitythe', 1),
 ('chemo', 1),
 ('handless', 1),
 ('fib', 1),
 ('iteris', 1),
 ('cureton', 1),
 ('mcm', 1),
 ('restocks', 1),
 ('cotter', 1),
 ('dpw', 1),
 ('evacuate', 1),
 ('fag', 1),
 ('rosion', 1),
 ('scratcher', 1),
 ('fizz', 1),
 ('diced', 1),
 ('hunker', 1),
 ('op